In [1]:
pip install pillow numpy


Note: you may need to restart the kernel to use updated packages.


In [1]:
from PIL import Image
import numpy as np

# Convert message to binary
def message_to_binary(message):
    return ''.join(format(ord(char), '08b') for char in message)

# Convert binary to message
def binary_to_message(binary):
    chars = [binary[i:i+8] for i in range(0, len(binary), 8)]
    return ''.join([chr(int(char, 2)) for char in chars])

In [2]:
# Hide the message in the image
def hide_message(image_path, secret_message, output_path="stego_image.png"):
    # Add a delimiter to detect end of message
    secret_message += "####"
    binary_message = message_to_binary(secret_message)

    img = Image.open(image_path)
    img = img.convert("RGB")
    data = np.array(img)

    flat_data = data.flatten()

    if len(binary_message) > len(flat_data):
        raise ValueError("Message is too long to hide in this image.")

    # Modify the LSB of each color value
    for i in range(len(binary_message)):
        flat_data[i] = (flat_data[i] & ~1) | int(binary_message[i])

    # Reshape and save the new image
    modified_data = flat_data.reshape(data.shape)
    new_img = Image.fromarray(modified_data.astype('uint8'), 'RGB')
    new_img.save(output_path)
    print(f"Message successfully hidden in {output_path}")

In [3]:
# Extract the hidden message from the image
def extract_message(image_path):
    img = Image.open(image_path)
    img = img.convert("RGB")
    data = np.array(img)
    flat_data = data.flatten()

    binary_message = ""
    for value in flat_data:
        binary_message += str(value & 1)

    # Split binary into 8-bit chunks
    chars = [binary_message[i:i+8] for i in range(0, len(binary_message), 8)]
    message = ""
    for char in chars:
        decoded_char = chr(int(char, 2))
        message += decoded_char
        if message.endswith("####"):
            break

    return message[:-4]

In [ ]:
if __name__ == "__main__":
    print("\n1. Hide a message in image")
    print("2. Extract a message from image")
    choice = input("Enter your choice (1 or 2): ").strip()

    if choice == "1":
        input_image = input("Enter input image path (e.g., input_image.png): ").strip()
        secret = input("Enter your secret message: ").strip()
        output_image = input("Enter output image path (e.g., stego.png): ").strip()
        hide_message(input_image, secret, output_image)

    elif choice == "2":
        stego_image = input("Enter stego image path: ").strip()
        hidden_text = extract_message(stego_image)
        print("\nHidden message extracted:\n", hidden_text)

    else:
        print("Invalid choice. Please enter 1 or 2.")



1. Hide a message in image
2. Extract a message from image


Enter your choice (1 or 2):  2
